This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***



 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

In [17]:
#pip install tensorflow==2.10.*
#pip install tensorflow-text==2.10.0
#pip install tensorflow_hub

**Classical Machine Learning - TF-IDF - All Features**

![CML TFIDF All](images\cml-tfidf-all.gif)

**Classical Machine Learning - TF-IDF - ExtraTreesClassifier Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-extra.gif)

**Classical Machine Learning - TF-IDF - InfoGain Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-infogain.gif)

**Classical Machine Learning - TF-IDF - SelectKBest Features**

![CML TFIDF ExtraTrees](images\cml-tfidf-selectkbest.gif)

**Classical Machine Learning - Word Embeddings - No Stopwords**

![CML TFIDF ExtraTrees](images\cml-we-swno.gif)

**Classical Machine Learning - Word Embeddings - Stopwords**

![CML TFIDF ExtraTrees](images\cml-we-swyes.gif)

In [18]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime
from datetime import timedelta
from tqdm import tqdm
import torchtext
from nltk.corpus import stopwords
import tensorflow_hub as hub

from sklearn.metrics import roc_auc_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, svm, naive_bayes
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# define data path
DATA_PATH = './obesity_data/'
RESULTS_PATH = './results/'
MODELS_PATH = './models/'

if os.path.exists(RESULTS_PATH) == False:
    os.mkdir(RESULTS_PATH)
if os.path.exists(MODELS_PATH) == False:
    os.mkdir(MODELS_PATH)

cachedStopWords = stopwords.words("english")

#Download info for USE
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

all_docs_df = pd.read_pickle(DATA_PATH + '/alldocs_df.pkl')
all_docs_df_ns = pd.read_pickle(DATA_PATH + '/alldocs_df_ns.pkl')
all_annot_df = pd.read_pickle(DATA_PATH + '/allannot_df.pkl')
all_df_expanded = pd.read_pickle(DATA_PATH + '/all_df_expanded.pkl')


In [19]:
all_df = pd.merge(all_docs_df,all_annot_df, on='id')
all_df_ns = pd.merge(all_docs_df_ns,all_annot_df, on='id')

disease_list = all_df['disease'].unique().tolist()
#disease_list = ['Asthma']
feature_list = ['All','ExtraTreeClassifier','SelectKBest','InfoGainAttributeVal']
embedding_list = ['GloVe', 'FastText', 'USE']
device = torch.device('cpu')

In [20]:
result_cols = ['Batch','Disease','Classifier','Feature', 'F1_MACRO', 'F1_MICRO', 'Total Run (secs)']

def write_to_file(file, batch_name, disease, clfr, feature,f1_macro,f1_micro,runtime_sec):
    #Pass TFIDF or Embeddings
    
    results_file = f'{RESULTS_PATH}CML_{file}_results.csv'

    if os.path.exists(results_file):
        results = pd.read_csv(results_file)
    else:
        results = pd.DataFrame(columns=result_cols)

    result = pd.DataFrame(columns=result_cols,data=[[batch_name, disease,clfr,feature,f1_macro,f1_micro,runtime_sec]])
    results = pd.concat([results,result])

    #Save results - overwrite so we can see progress
    results.to_csv(results_file, index=False)

In [21]:
for index, entry in enumerate(all_df['tok_lem_text']):
    Final_words = []
    for word in entry:
        Final_words.append(word)
    all_df.loc[index, 'text_final'] = str(Final_words)

In [22]:
for index, entry in enumerate(all_df_ns['tok_lem_text']):
    Final_words = []
    for word in entry:
        Final_words.append(word)
    all_df_ns.loc[index, 'text_final'] = str(Final_words)

In [23]:
def performSVM(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):
    start_time = time.time()
    SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
    SVM.fit(Train_X_Tfidf, y_train)

    predictions_SVM = SVM.predict(Test_X_Tfidf)
    end_time = time.time()
    runtime_sec = end_time-start_time

    #f1 = f1_score(y_test, predictions_SVM)
    f1_macro = f1_score(y_test, predictions_SVM,average='macro')
    f1_micro = f1_score(y_test, predictions_SVM,average='micro')

    #print("SVM - ", disease, ": f1-score", f1)
    print("SVM - ", feature, disease, ": f1-macro", f1_macro)
    print("SVM - ", feature, disease, ": f1-micro", f1_micro)
    
    write_to_file(file, batch_name, disease, "SVM", feature,f1_macro, f1_micro, runtime_sec)

    return f1_macro, f1_micro

In [24]:
def performKNN(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):

    start_time = time.time()
    knn1 = KNeighborsClassifier(n_neighbors=1)
    clf1 = knn1.fit(Train_X_Tfidf, y_train)
    # predict the labels on validation dataset
    predictions_KNN1 = clf1.predict(Test_X_Tfidf)
    end_time = time.time()
    runtime_sec1 = end_time-start_time

    start_time = time.time()
    knn5 = KNeighborsClassifier(n_neighbors=5)
    clf5 = knn5.fit(Train_X_Tfidf, y_train)
    # predict the labels on validation dataset
    predictions_KNN5 = clf5.predict(Test_X_Tfidf)
    end_time = time.time()
    runtime_sec5 = end_time-start_time


    #auroc = roc_auc_score(truth, pred[:,1])
    #f1 = f1_score(y_test, predictions_KNN)
    f1_macro1 = f1_score(y_test, predictions_KNN1,average='macro')
    f1_macro5 = f1_score(y_test, predictions_KNN5,average='macro')
    f1_micro1 = f1_score(y_test, predictions_KNN1,average='micro')
    f1_micro5 = f1_score(y_test, predictions_KNN5,average='micro')

    #print("KNN - ", disease, ": f1-score", f1)
    print("KNN 1 k - ", feature, disease, ": f1-macro", f1_macro1)
    print("KNN 1 k - ", feature, disease, ": f1-micro", f1_micro1)
    print("KNN 5 k - ", feature, disease, ": f1-macro", f1_macro5)
    print("KNN 5 k - ", feature, disease, ": f1-micro", f1_micro5)
    
    write_to_file(file, batch_name, disease, "KNN n=1", feature,f1_macro1,f1_micro1, runtime_sec1)
    write_to_file(file, batch_name, disease, "KNN n=5", feature,f1_macro1,f1_micro1, runtime_sec5)

    return f1_macro1, f1_micro1, f1_macro5, f1_micro5

In [25]:
def performNB(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):
    start_time = time.time()
    scaler = MinMaxScaler()
    scaler.fit(Train_X_Tfidf)
    
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(scaler.transform(Train_X_Tfidf),y_train)

    # predict the labels on validation dataset
    predictions_NB = Naive.predict(Test_X_Tfidf)
    end_time = time.time()
    runtime_sec = end_time-start_time

    #f1 = f1_score(y_test, predictions_NB)
    f1_macro = f1_score(y_test, predictions_NB,average='macro')
    f1_micro = f1_score(y_test, predictions_NB,average='micro')

    #print("NB - ", disease, ": f1-score", f1)
    print("NB - ", feature, disease, ": f1-macro", f1_macro)
    print("NB - ", feature, disease, ": f1-micro", f1_micro)

    write_to_file(file, batch_name, disease, "Naive Bayes", feature,f1_macro, f1_micro, runtime_sec)
    
    return f1_macro, f1_micro

In [26]:
def performRF(Train_X_Tfidf, Test_X_Tfidf, y_train, y_test, feature):
    start_time = time.time()

    classifier=RandomForestClassifier(n_estimators =400,criterion="entropy",random_state =0)
    classifier.fit(Train_X_Tfidf,y_train)

    # predict the labels on validation dataset
    predictions_RF = classifier.predict(Test_X_Tfidf)
    end_time = time.time()
    runtime_sec = end_time-start_time

    #f1 = f1_score(y_test, predictions_RF)
    f1_macro = f1_score(y_test, predictions_RF,average='macro')
    f1_micro = f1_score(y_test, predictions_RF,average='micro')

    #print("RF - ", disease, ": f1-score", f1)
    print("RF - ", feature, disease, ": f1-macro", f1_macro)
    print("RF - ", feature, disease, ": f1-micro", f1_micro)
    
    write_to_file(file, batch_name, disease, "Random Forest", feature,f1_macro, f1_micro, runtime_sec)

    return f1_macro, f1_micro

In [27]:
def vectorize_batch_GloVe(X_Train):
    embedding_size_used = 300
    vec = torchtext.vocab.GloVe(name='6B', dim=embedding_size_used)
    
    X =  np.zeros((X_Train.shape[0], embedding_size_used * len(X_Train.iloc[0])))
    
    for i in range(len(X_Train)):
        vectors = vec.get_vecs_by_tokens(X_Train.iloc[i]).float().numpy()
        
        X[i,:] = vectors.flatten()
        
    return X

In [28]:
def vectorize_batch_FastText(X_Train):
    embedding_size_used = 300
    vec = torchtext.vocab.FastText()
    
    X =  np.zeros((X_Train.shape[0], embedding_size_used * len(X_Train.iloc[0])))
    
    for i in range(len(X_Train)):
        vectors = vec.get_vecs_by_tokens(X_Train.iloc[i]).float().numpy()
        
        X[i,:] = vectors.flatten()
        
    return X

In [29]:
def vectorize_batch_USE(X_Train):
    embedding_size_used = 512
    
    X =  np.zeros((X_Train.shape[0], embedding_size_used * len(X_Train.iloc[0])))
    
    for i in range(len(X_Train)):
        tensor_flow_vectors = embed(X_Train.iloc[i])
        array_vectors = tensor_flow_vectors.numpy()
        
        X[i,:] = array_vectors.flatten()
        
    return X

In [30]:
def generateXTrainAndTest(X_train, X_test, Tfidf_vect):
    X_train_values_list = Tfidf_vect.fit_transform(X_train).toarray()
    X_training = pd.DataFrame(X_train_values_list, columns=Tfidf_vect.get_feature_names_out())
    X_training = np.asarray(X_training, dtype=float)
    X_training = torch.from_numpy(X_training).to(device)

    X_test_values_list = Tfidf_vect.transform(X_test).toarray()
    X_testing = pd.DataFrame(X_test_values_list, columns=Tfidf_vect.get_feature_names_out())
    X_testing = np.asarray(X_testing, dtype=float)
    X_testing = torch.from_numpy(X_testing).to(device)  
    
    return X_training, X_testing

In [31]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV, RFE
from sklearn.tree import ExtraTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import f_classif, mutual_info_classif
from statistics import mean
from sklearn.preprocessing import MinMaxScaler

def getVocab(X_train, y_train, feature, max_tokens):
 
    ## Step 1: Determine the Initial Vocabulary
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    vocab = list(tokenizer.word_index.keys())

    ## Step 2: Create term  matrix
    vectors = tokenizer.texts_to_matrix(X_train, mode='count')

    ## Do feature selection on term matrix (column headers are words)
    X = vectors
    y = y_train

    ##Choose algorithm
    if feature == 'SelectKBest':
        selector = SelectKBest(score_func=f_classif, k=max_tokens).fit(X,y)
    else: 
        if feature == 'InfoGainAttributeVal':
            #This should be similar to the InfoGain?
            selector = SelectKBest(score_func=mutual_info_classif, k=max_tokens).fit(X,y)
        else:
            #default to ExtraTreeClassifier
            estimator = ExtraTreeClassifier(random_state = seed)
            selector = SelectFromModel(estimator, max_features = max_tokens)
            selector = selector.fit(X, y)

    support_idx = selector.get_support(True)
    tokenizer2 = Tokenizer()
    tokenizer2.fit_on_texts([vocab[i-1].replace("'","") for i in support_idx])
    new_vocab = list(tokenizer2.word_index.keys())

    return new_vocab

# Feature Selection

In [32]:
file = 'TFIDF'
result_time = datetime.datetime.now()
result_name = result_time.strftime("%Y-%m-%d-%H-%M-%S")
batch_name = f'CML_tfidf_results_{result_name}'

Encoder = LabelEncoder()
Tfidf_vect = TfidfVectorizer(max_features=600)
Tfidf_vect_NS = TfidfVectorizer(max_features = 600, stop_words = cachedStopWords)

svm_f1_micro_scores = []
svm_f1_macro_scores = []
knn1_f1_micro_scores = []
knn1_f1_macro_scores = []
knn5_f1_micro_scores = []
knn5_f1_macro_scores = []
nb_f1_micro_scores = []
nb_f1_macro_scores = []
rf_f1_micro_scores = []
rf_f1_macro_scores = []

max_tokens = 600

for _,feature in enumerate(feature_list):
    for _,disease in enumerate(disease_list):
        disease_data_df = all_df[all_df['disease'] == disease]

        X_train, X_test, y_train, y_test = train_test_split(disease_data_df['text_final'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
        
        if feature != 'All':
            vocab = getVocab(X_train,y_train, feature, max_tokens)
            Tfidf_vect = TfidfVectorizer(max_features=max_tokens,vocabulary = vocab)
        else:
            Tfidf_vect = TfidfVectorizer(max_features=max_tokens)
  
        
        X_train_values_list = Tfidf_vect.fit_transform(X_train).toarray()
        X_training = pd.DataFrame(X_train_values_list, columns=Tfidf_vect.get_feature_names_out())
        X_training = np.asarray(X_training, dtype=float)
        X_training = torch.from_numpy(X_training).to(device)

        X_test_values_list = Tfidf_vect.transform(X_test).toarray()
        X_testing = pd.DataFrame(X_test_values_list, columns=Tfidf_vect.get_feature_names_out())
        X_testing = np.asarray(X_testing, dtype=float)
        X_testing = torch.from_numpy(X_testing).to(device)
        
        #tokens_to_use = X_training.shape[1]

        Train_Y  = Encoder.fit_transform(y_train)
        Test_Y  = Encoder.fit_transform(y_test)

        svm_f1_macro, svm_f1_micro = performSVM(X_training, X_testing, Train_Y, Test_Y, feature)
        svm_f1_macro_scores.append(svm_f1_macro)
        svm_f1_micro_scores.append(svm_f1_micro)

        knn_f1_macro1, knn_f1_micro1, knn_f1_macro5, knn_f1_micro5 = performKNN(X_training, X_testing, Train_Y, Test_Y, feature)
        knn1_f1_macro_scores.append(knn_f1_macro1)
        knn1_f1_micro_scores.append(knn_f1_micro1)
        knn5_f1_macro_scores.append(knn_f1_macro5)
        knn5_f1_micro_scores.append(knn_f1_micro5)

        nb_f1_macro, nb_f1_micro = performNB(X_training, X_testing, Train_Y, Test_Y, feature)
        nb_f1_macro_scores.append(nb_f1_macro)
        nb_f1_micro_scores.append(nb_f1_micro)

        rf_f1_macro, rf_f1_micro = performRF(X_training, X_testing, Train_Y, Test_Y, feature)
        rf_f1_macro_scores.append(rf_f1_macro)
        rf_f1_micro_scores.append(rf_f1_micro)
        
    print("Average SVM - ", feature, ": f1-micro", mean(svm_f1_micro_scores))
    print("Average SVM - ", feature, ": f1-macro", mean(svm_f1_macro_scores))
    
    print("Average KNN 1 - ", feature, ": f1-micro", mean(knn1_f1_micro_scores))
    print("Average KNN 1 - ", feature, ": f1-macro", mean(knn1_f1_macro_scores))
    print("Average KNN 5 - ", feature, ": f1-micro", mean(knn5_f1_micro_scores))
    print("Average KNN 5 - ", feature, ": f1-macro", mean(knn5_f1_macro_scores))
    
    print("Average NB - ", feature, ": f1-micro", mean(nb_f1_micro_scores))
    print("Average NB - ", feature, ": f1-macro", mean(nb_f1_macro_scores))
    
    print("Average RF - ", feature, ": f1-micro", mean(rf_f1_micro_scores))
    print("Average RF - ", feature, ": f1-macro", mean(rf_f1_macro_scores))
        

SVM -  All Asthma : f1-macro 0.609237775127399
SVM -  All Asthma : f1-micro 0.9198113207547169
KNN 1 k -  All Asthma : f1-macro 0.5623043623043623
KNN 1 k -  All Asthma : f1-micro 0.8537735849056604
KNN 5 k -  All Asthma : f1-macro 0.5846627483907081
KNN 5 k -  All Asthma : f1-micro 0.9009433962264151
NB -  All Asthma : f1-macro 0.6466666666666667
NB -  All Asthma : f1-micro 0.9245283018867925


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

RF -  All Asthma : f1-macro 0.6466666666666667
RF -  All Asthma : f1-micro 0.9245283018867925
Average SVM -  All : f1-micro 0.9198113207547169
Average SVM -  All : f1-macro 0.609237775127399
Average KNN 1 -  All : f1-micro 0.8537735849056604
Average KNN 1 -  All : f1-macro 0.5623043623043623
Average KNN 5 -  All : f1-micro 0.9009433962264151
Average KNN 5 -  All : f1-macro 0.5846627483907081
Average NB -  All : f1-micro 0.9245283018867925
Average NB -  All : f1-macro 0.6466666666666667
Average RF -  All : f1-micro 0.9245283018867925
Average RF -  All : f1-macro 0.6466666666666667
SVM -  ExtraTreeClassifier Asthma : f1-macro 0.6002514142049027
SVM -  ExtraTreeClassifier Asthma : f1-micro 0.8726415094339622
KNN 1 k -  ExtraTreeClassifier Asthma : f1-macro 0.605169591963429
KNN 1 k -  ExtraTreeClassifier Asthma : f1-micro 0.8443396226415094
KNN 5 k -  ExtraTreeClassifier Asthma : f1-macro 0.5484392214469336
KNN 5 k -  ExtraTreeClassifier Asthma : f1-micro 0.8632075471698113
NB -  ExtraTre

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

RF -  ExtraTreeClassifier Asthma : f1-macro 0.5750286368843069
RF -  ExtraTreeClassifier Asthma : f1-micro 0.8679245283018869
Average SVM -  ExtraTreeClassifier : f1-micro 0.8962264150943395
Average SVM -  ExtraTreeClassifier : f1-macro 0.6047445946661508
Average KNN 1 -  ExtraTreeClassifier : f1-micro 0.8490566037735849
Average KNN 1 -  ExtraTreeClassifier : f1-macro 0.5837369771338956
Average KNN 5 -  ExtraTreeClassifier : f1-micro 0.8820754716981132
Average KNN 5 -  ExtraTreeClassifier : f1-macro 0.5665509849188208
Average NB -  ExtraTreeClassifier : f1-micro 0.8938679245283019
Average NB -  ExtraTreeClassifier : f1-macro 0.5710145284412707
Average RF -  ExtraTreeClassifier : f1-micro 0.8962264150943398
Average RF -  ExtraTreeClassifier : f1-macro 0.6108476517754868


...

SVM -  SelectKBest Asthma : f1-macro 0.8923311325545962
SVM -  SelectKBest Asthma : f1-micro 0.9433962264150944
KNN 1 k -  SelectKBest Asthma : f1-macro 0.6372192513368984
KNN 1 k -  SelectKBest Asthma : f1-micro 0.8490566037735849
KNN 5 k -  SelectKBest Asthma : f1-macro 0.6290987076020091
KNN 5 k -  SelectKBest Asthma : f1-micro 0.8537735849056604
NB -  SelectKBest Asthma : f1-macro 0.5004776157422148
NB -  SelectKBest Asthma : f1-micro 0.8254716981132075


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

RF -  SelectKBest Asthma : f1-macro 0.8110853680270896
RF -  SelectKBest Asthma : f1-micro 0.9056603773584906
Average SVM -  SelectKBest : f1-micro 0.9119496855345912
Average SVM -  SelectKBest : f1-macro 0.7006067739622993
Average KNN 1 -  SelectKBest : f1-micro 0.8490566037735849
Average KNN 1 -  SelectKBest : f1-macro 0.6015644018682299
Average KNN 5 -  SelectKBest : f1-micro 0.8726415094339622
Average KNN 5 -  SelectKBest : f1-macro 0.5874002258132169
Average NB -  SelectKBest : f1-micro 0.8710691823899371
Average NB -  SelectKBest : f1-macro 0.547502224208252
Average RF -  SelectKBest : f1-micro 0.89937106918239
Average RF -  SelectKBest : f1-macro 0.6775935571926878
SVM -  InfoGainAttributeVal Asthma : f1-macro 0.8885223959080262
SVM -  InfoGainAttributeVal Asthma : f1-micro 0.9481132075471698
KNN 1 k -  InfoGainAttributeVal Asthma : f1-macro 0.6768122798717342
KNN 1 k -  InfoGainAttributeVal Asthma : f1-micro 0.8632075471698113
KNN 5 k -  InfoGainAttributeVal Asthma : f1-macro 0

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

RF -  InfoGainAttributeVal Asthma : f1-macro 0.8479850853291266
RF -  InfoGainAttributeVal Asthma : f1-micro 0.9292452830188679
Average SVM -  InfoGainAttributeVal : f1-micro 0.9209905660377358
Average SVM -  InfoGainAttributeVal : f1-macro 0.747585679448731
Average KNN 1 -  InfoGainAttributeVal : f1-micro 0.8525943396226415
Average KNN 1 -  InfoGainAttributeVal : f1-macro 0.620376371369106
Average KNN 5 -  InfoGainAttributeVal : f1-micro 0.8738207547169812
Average KNN 5 -  InfoGainAttributeVal : f1-macro 0.608133901417329
Average NB -  InfoGainAttributeVal : f1-micro 0.8702830188679246
Average NB -  InfoGainAttributeVal : f1-macro 0.5594540474853409
Average RF -  InfoGainAttributeVal : f1-micro 0.9068396226415094
Average RF -  InfoGainAttributeVal : f1-macro 0.7201914392267975


# Word Embeddings

In [33]:
file = 'Embeddings'
result_time = datetime.datetime.now()
result_name = result_time.strftime("%Y-%m-%d-%H-%M-%S")
batch_name = f'CML_tfidf_results_{result_name}'

Encoder = LabelEncoder()
Tfidf_vect = TfidfVectorizer(max_features=600)
Tfidf_vect_NS = TfidfVectorizer(max_features = 600, stop_words = cachedStopWords)

svm_f1_micro_scores = []
svm_f1_macro_scores = []
knn1_f1_micro_scores = []
knn1_f1_macro_scores = []
knn5_f1_micro_scores = []
knn5_f1_macro_scores = []
nb_f1_micro_scores = []
nb_f1_macro_scores = []
rf_f1_micro_scores = []
rf_f1_macro_scores = []

max_tokens = 600

for _, embedding in enumerate(embedding_list):
    for _,disease in enumerate(disease_list):
        disease_data_df = all_df_expanded [all_df_expanded ['disease'] == disease]

        if embedding == 'GloVe':
            X_train, X_test, y_train, y_test = train_test_split(disease_data_df['vector_tokenized'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
            X_train = vectorize_batch_GloVe(X_train)
            X_test = vectorize_batch_GloVe(X_test)
        if embedding == 'FastText':
            X_train, X_test, y_train, y_test = train_test_split(disease_data_df['vector_tokenized'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
            X_train = vectorize_batch_FastText(X_train)
            X_test = vectorize_batch_FastText(X_test)
        if embedding == 'USE':
            X_train, X_test, y_train, y_test = train_test_split(disease_data_df['sentence_tokenized'], disease_data_df['judgment'], test_size=0.20, shuffle=True)
            X_train = vectorize_batch_USE(X_train)
            X_test = vectorize_batch_USE(X_test)

        Train_Y  = Encoder.fit_transform(y_train)
        Test_Y  = Encoder.fit_transform(y_test)

        svm_f1_macro, svm_f1_micro = performSVM(X_train, X_test, Train_Y, Test_Y, embedding)
        svm_f1_macro_scores.append(svm_f1_macro)
        svm_f1_micro_scores.append(svm_f1_micro)

        knn_f1_macro1, knn_f1_micro1, knn_f1_macro5, knn_f1_micro5 = performKNN(X_train, X_test, Train_Y, Test_Y, embedding)
        knn1_f1_macro_scores.append(knn_f1_macro1)
        knn1_f1_micro_scores.append(knn_f1_micro1)
        knn5_f1_macro_scores.append(knn_f1_macro5)
        knn5_f1_micro_scores.append(knn_f1_micro5)

        nb_f1_macro, nb_f1_micro = performNB(X_train, X_test, Train_Y, Test_Y, embedding)
        nb_f1_macro_scores.append(nb_f1_macro)
        nb_f1_micro_scores.append(nb_f1_micro)

        rf_f1_macro, rf_f1_micro = performRF(X_train, X_test, Train_Y, Test_Y, embedding)
        rf_f1_macro_scores.append(rf_f1_macro)
        rf_f1_micro_scores.append(rf_f1_micro)
        
    print("Average SVM - ", embedding, ": f1-micro", mean(svm_f1_micro_scores))
    print("Average SVM - ", embedding, ": f1-macro", mean(svm_f1_macro_scores))

    print("Average KNN 1 - ", embedding, ": f1-micro", mean(knn1_f1_micro_scores))
    print("Average KNN 1 - ", embedding, ": f1-macro", mean(knn1_f1_macro_scores))
    print("Average KNN 5 - ", embedding, ": f1-micro", mean(knn5_f1_micro_scores))
    print("Average KNN 5 - ", embedding, ": f1-macro", mean(knn5_f1_macro_scores))

    print("Average NB - ", embedding, ": f1-micro", mean(nb_f1_micro_scores))
    print("Average NB - ", embedding, ": f1-macro", mean(nb_f1_macro_scores))

    print("Average RF - ", embedding, ": f1-micro", mean(rf_f1_micro_scores))
    print("Average RF - ", embedding, ": f1-macro", mean(rf_f1_macro_scores))

SVM -  GloVe Asthma : f1-macro 0.4642857142857143
SVM -  GloVe Asthma : f1-micro 0.8666666666666667


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

KNN 1 k -  GloVe Asthma : f1-macro 0.4642857142857143
KNN 1 k -  GloVe Asthma : f1-micro 0.8666666666666667
KNN 5 k -  GloVe Asthma : f1-macro 0.4642857142857143
KNN 5 k -  GloVe Asthma : f1-micro 0.8666666666666667
NB -  GloVe Asthma : f1-macro 0.524603836530442
NB -  GloVe Asthma : f1-micro 0.8416666666666667
RF -  GloVe Asthma : f1-macro 0.4642857142857143
RF -  GloVe Asthma : f1-micro 0.8666666666666667
Average SVM -  GloVe : f1-micro 0.8666666666666667
Average SVM -  GloVe : f1-macro 0.4642857142857143
Average KNN 1 -  GloVe : f1-micro 0.8666666666666667
Average KNN 1 -  GloVe : f1-macro 0.4642857142857143
Average KNN 5 -  GloVe : f1-micro 0.8666666666666667
Average KNN 5 -  GloVe : f1-macro 0.4642857142857143
Average NB -  GloVe : f1-micro 0.8416666666666667
Average NB -  GloVe : f1-macro 0.524603836530442
Average RF -  GloVe : f1-micro 0.8666666666666667
Average RF -  GloVe : f1-macro 0.4642857142857143
SVM -  FastText Asthma : f1-macro 0.47019867549668876
SVM -  FastText Asthma

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

KNN 1 k -  FastText Asthma : f1-macro 0.47019867549668876
KNN 1 k -  FastText Asthma : f1-micro 0.8875
KNN 5 k -  FastText Asthma : f1-macro 0.47019867549668876
KNN 5 k -  FastText Asthma : f1-micro 0.8875
NB -  FastText Asthma : f1-macro 0.2671563923156113
NB -  FastText Asthma : f1-micro 0.2708333333333333
RF -  FastText Asthma : f1-macro 0.47019867549668876
RF -  FastText Asthma : f1-micro 0.8875
Average SVM -  FastText : f1-micro 0.8770833333333333
Average SVM -  FastText : f1-macro 0.46724219489120156
Average KNN 1 -  FastText : f1-micro 0.8770833333333333
Average KNN 1 -  FastText : f1-macro 0.46724219489120156
Average KNN 5 -  FastText : f1-micro 0.8770833333333333
Average KNN 5 -  FastText : f1-macro 0.46724219489120156
Average NB -  FastText : f1-micro 0.55625
Average NB -  FastText : f1-macro 0.3958801144230266
Average RF -  FastText : f1-micro 0.8770833333333333
Average RF -  FastText : f1-macro 0.46724219489120156
SVM -  USE Asthma : f1-macro 0.6133118506583699
SVM -  USE A

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdim

KNN 1 k -  USE Asthma : f1-macro 0.636094361578335
KNN 1 k -  USE Asthma : f1-micro 0.8208333333333333
KNN 5 k -  USE Asthma : f1-macro 0.5066392577365408
KNN 5 k -  USE Asthma : f1-micro 0.8458333333333333
NB -  USE Asthma : f1-macro 0.4606741573033708
NB -  USE Asthma : f1-micro 0.8541666666666666
RF -  USE Asthma : f1-macro 0.6133118506583699
RF -  USE Asthma : f1-micro 0.8791666666666667
Average SVM -  USE : f1-micro 0.8777777777777778
Average SVM -  USE : f1-macro 0.5159320801469243
Average KNN 1 -  USE : f1-micro 0.8583333333333333
Average KNN 1 -  USE : f1-macro 0.5235262504535794
Average KNN 5 -  USE : f1-micro 0.8666666666666667
Average KNN 5 -  USE : f1-macro 0.4803745491729813
Average NB -  USE : f1-micro 0.6555555555555556
Average NB -  USE : f1-macro 0.41747812871647466
Average RF -  USE : f1-micro 0.8777777777777778
Average RF -  USE : f1-macro 0.5159320801469243
